In [1]:
from custom_dataloader import import_data_loader
import custom_dataloader
from custom_model import retina_fpn_swin_l
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.optim as optim
import torch.nn as nn
from torchvision.ops import focal_loss
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from torch.utils.data import DataLoader

/home/user304/anaconda3/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:

###################################### transformer ##############################
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
transformer = A.Compose([
    A.Resize(512, 512),
    A.Flip(),
    A.OneOf([
        A.Rotate(limit=(0, 90), p=0.7),
        A.Compose([
            A.HorizontalFlip(always_apply=True),
            A.Rotate(limit=(-90, 0), p=0.7)
        ])
    ], p=0.7),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=(
            0, 0.2), contrast_limit=(0, 0.3), p=0.7),
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.7),
        A.Sharpen(alpha=(0.2, 0.8), lightness=(0.5, 1.0), p=0.7)
    ], p=0.7),
    A.Normalize(mean=mean, std=std,
                max_pixel_value=255.0),
    ToTensorV2()
], bbox_params=A.BboxParams(format='coco', label_fields=['class_ids']))

val_transformer = A.Compose([
    A.Resize(height=512, width=512),
    A.Normalize(mean=mean, std=std,
                max_pixel_value=255.0),
    ToTensorV2(),
],
    bbox_params=A.BboxParams(format='coco', label_fields=['class_ids']),
)

##########################################################################################
device = torch.device('cuda')

train_dataset = custom_dataloader.CustomDataset(
    '../info/train.json', transformer=transformer)
train_dataloader = import_data_loader(
    '../info/train.json', transformer=transformer)
val_dataset = custom_dataloader.TestCustomDataset(
    '../info/test.json', transformer=val_transformer)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=8,
                            collate_fn=custom_dataloader.collate_fn)


de_std = tuple(std * 255 for std in std)
de_mean = tuple(mean * 255 for mean in mean)


model = retina_fpn_swin_l()

optimizer = optim.AdamW(model.parameters(),
                        lr=0.0001 / 8,
                        betas=(0.9, 0.999),
                        weight_decay=0.05,
                        )

model = model.cuda()
model.load_state_dict(torch.load('./test3.pth'))


epochs = 50
cls_losses = 0
bbox_losses = 0
total_losses = 0
# for epoch in range(1, epochs + 1):
#     for i, (img_metas, images, bboxes, labels) in enumerate(train_dataloader):
#         images = images.cuda()
#         model.train()

#         bboxes = [bbox.cuda() for bbox in bboxes]
#         labels = [label.cuda() for label in labels]
#         outputs = model(images)
#         losses = model.bbox_head.loss(
#             cls_scores=outputs[0],
#             bbox_preds=outputs[1],
#             gt_bboxes=bboxes,
#             gt_labels=labels,
#             img_metas=img_metas
#         )

#         loss_cls_total = losses['loss_cls'][0] + \
#             losses['loss_cls'][1] + losses['loss_cls'][2]
#         loss_bbox_total = losses['loss_bbox'][0] + \
#             losses['loss_bbox'][1] + losses['loss_bbox'][2]
#         total_loss = loss_cls_total + loss_bbox_total
#         total_loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         cls_losses += loss_cls_total
#         bbox_losses += loss_bbox_total
#         total_losses += total_loss
#         if i % 100 == 0 and i > 0:
#             print(f'[{i} / {len(train_dataloader)}]')
#             print(
#                 f'step : {i}\nloss_bbox : {bbox_losses / 100}  loss_cls_total : {cls_losses /100}  total_loss : {total_losses / 100}')
#             cls_losses = 0
#             bbox_losses = 0
#             total_losses = 0
    # if epoch % 10 == 0:
    #     cls_losses = 0
    #     bbox_losses = 0
    #     total_losses = 0
    #     with torch.no_grad():
    #         model.eval()
    #         for j, (img_metas, images, bboxes, labels) in enumerate(val_dataloader):
    #             bboxes = [bbox.cuda() for bbox in bboxes]
    #             labels = [label.cuda() for label in labels]
    #             outputs = model(images)
    #             losses = model.bbox_head.loss(
    #                 cls_scores=outputs[0],
    #                 bbox_preds=outputs[1],
    #                 gt_bboxes=bboxes,
    #                 gt_labels=labels,
    #                 img_metas=img_metas
    #             )
    #             loss_cls_total = losses['loss_cls'][0] + \
    #                 losses['loss_cls'][1] + losses['loss_cls'][2]
    #             loss_bbox_total = losses['loss_bbox'][0] + \
    #                 losses['loss_bbox'][1] + losses['loss_bbox'][2]
    #             total_loss = loss_cls_total + loss_bbox_total
    #             cls_losses += loss_cls_total
    #             bbox_losses += loss_bbox_total
    #             total_losses += total_loss

    #             bbox_preds = model.bbox_head.get_bboxes(
    #                 cls_scores=outputs[0],
    #                 bbox_preds=outputs[1],
    #                 img_metas=img_metas
    #             )
    #         print(f'step : {i}\nloss_bbox : {bbox_losses / len(val_dataloader)}   \
    #                 loss_cls_total : {cls_losses /len(val_dataloader)}  total_loss : {total_losses / len(val_dataloader)}')
    # print(f'epoch : {epoch}')
    # print('-' * 30)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [3]:
import os 
from pycocotools.coco import COCO
coco_gt = COCO('../info/test.json')
img_name_to_id = {os.path.basename(label_info['file_name']).split('.jpg')[0]: label_info['id']
                               for label_info in coco_gt.loadImgs(coco_gt.getImgIds())}

val_dataset = custom_dataloader.TestCustomDataset(
    '../info/test.json', transformer=val_transformer)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=8,
                            collate_fn=custom_dataloader.collate_fn)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
cls_losses = 0
bbox_losses = 0
total_losses = 0

preds_list = []
with torch.no_grad():
    model.eval()
    for j, (img_metas, images, bboxes, labels) in enumerate(val_dataloader):
        images = images.cuda()
        bboxes = [bbox.cuda() for bbox in bboxes]
        labels = [label.cuda() for label in labels]
        outputs = model(images)
        losses = model.bbox_head.loss(
            cls_scores=outputs[0],
            bbox_preds=outputs[1],
            gt_bboxes=bboxes,
            gt_labels=labels,
            img_metas=img_metas
        )
        loss_cls_total = losses['loss_cls'][0] + \
            losses['loss_cls'][1] + losses['loss_cls'][2]
        loss_bbox_total = losses['loss_bbox'][0] + \
            losses['loss_bbox'][1] + losses['loss_bbox'][2]
        total_loss = loss_cls_total + loss_bbox_total
        cls_losses += loss_cls_total
        bbox_losses += loss_bbox_total
        total_losses += total_loss

        preds = model.bbox_head.get_bboxes(
            cls_scores=outputs[0],
            bbox_preds=outputs[1],
            img_metas=img_metas ,
            rescale = True
        )
        preds = [(pred[0].cpu().numpy(),pred[1].cpu().numpy())for pred in preds]
        preds_list.extend(zip( [img_name_to_id[os.path.basename(img_meta['filename']).split('.')[0]] for img_meta in img_metas], preds))
        # break
    print(f'step : {j}\nloss_bbox : {bbox_losses / len(val_dataloader)}   \
            loss_cls_total : {cls_losses /len(val_dataloader)}  total_loss : {total_losses / len(val_dataloader)}')
    

RuntimeError: Unable to find a valid cuDNN algorithm to run convolution

In [5]:
import utils

In [6]:
result = [{'image_id' : pred_list[0],
    'bbox' : utils.type_converter.cv2_to_coco(pred_list[1][0][i][:-1].tolist() , dim =1),
    'score' : pred_list[1][0][i][-1],
    'category_id' : pred_list[1][1][i]
    }
    for pred_list in preds_list 
    for i in range(len(pred_list[1][1]))]

In [8]:
len(result)

33998

In [9]:
import json
with open('result.json', "w") as json_file:
    json.dump(result, json_file, indent=4 , cls = utils.type_converter.NpEncoder)

In [25]:
coco_pred = coco_gt.loadRes('result.json')

Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!


In [49]:
coco_pred.loadAnns(coco_pred.getAnnIds(imgIds = coco_pred.getImgIds()))[-18]

{'image_id': 28485,
 'bbox': [496.8477478027344, 0.0, 15.152252197265625, 16.7343807220459],
 'score': 0.6105105876922607,
 'category_id': 2,
 'segmentation': [[496.8477478027344,
   0.0,
   496.8477478027344,
   16.7343807220459,
   512.0,
   16.7343807220459,
   512.0,
   0.0]],
 'area': 253.56355706549948,
 'id': 1685,
 'iscrowd': 0}

In [ ]:
coco_gt.loadAnns()

In [50]:
coco_pred.loadAnns()

In [26]:
coco_pred.getAnnIds()

In [56]:
from pycocotools.cocoeval import COCOeval

In [65]:
coco_pred[0]

TypeError: 'COCO' object is not subscriptable

In [57]:
cocoEval = COCOeval(coco_gt, coco_pred, iouType='bbox')
cocoEval.evaluate()
cocoEval.accumulate()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.30s).
Accumulating evaluation results...
DONE (t=0.06s).


In [60]:
cocoEval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
